In [1]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [4]:
# Cell 3

wallet = KeyWallet.load("../../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../../keystores/balanced_test.json", key_data)
print(icon_service.get_balance(wallet.get_address())/10**18)
print(icon_service.get_balance(btest_wallet.get_address())/10**18)

780851236.8012704
4177761.8313104


In [5]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
780851236.8012704


In [6]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
4177761.8313104


In [7]:
user1 = KeyWallet.load("../../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

197999.9138008
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
208479.8202492
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [8]:
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxbbd36ca8d91aefbe1060c3be62fed4d1b848ca85'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxd8e05c1280bc2c32bf53ff61f3bb2e2ecc7d6df5'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx7c617e3fca4ba06b6ad203ce113245ae96a9d91e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx70cd5c86f0182d5ac0bd224562d929cd968d9132'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxeb91bc377b0620356787d9c9eb68152eb0c62d8a'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxfd7511ece084744154fed19dc34732681ad078e6'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9a3161c778eee2d5758371d3548c5599f76704ec'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx238cd1a1e3a9702d6c9c6dc130719472164db376'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx2780eeb8c800ac9886786baae281c3d23bb832fc'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxcff8bf80ab213fa9bbb350636a4d68f5cb4fd9c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx4c1beaa71b9377100c810c46059ddf5f3da37602'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx3825a86d52c5baf188ff29aa6a7fc2467e285885'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx140b49ea041457ebc4cd5e199f5723916bb50021'}}

In [9]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    print(icon_service.call(call))
    return icon_service.call(call)


In [10]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print(get_scores_json(contracts))

../../core_contracts/governance.zip
Status: 1

../../core_contracts/loans.zip
Status: 1

../../core_contracts/dividends.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/reserve.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/daofund.zip


...

Retrying in 1 seconds...
Status: 1

../../token_contracts/bwt.zip
Status: 1

../../core_contracts/staking.zip
Status: 1

../../token_contracts/sicx.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

------------------------------------------------------------------------------------------------------------------
Calling setSicxAddress, with parameters {'_address': 'cxb2d986916fe48a424554526baf61c0451222a067'} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1', 'staking': 'cx252d78d074053dc19652aa25ad659824fdc31d7e', 'dividends': 'cxfa5b615097d9ad7b52ea10734c70f4b4a636ff1a', 'reserve': 'cx937a4d5f5cc89d3028e91f2ef44355ed7c54357c', 'daofund': 'cxc567be9276434f34d285b671e7c9360e63ac2746', 'rewards': 'cx593e188212e9c29fa04473d9af4ffe

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1', 'indexed': ['AssetAdded(Address,str,bool)', 'cxb2d986916fe48a424554526baf61c0451222a067', 'sICX', '0x1'], 'data': []} 

{'scoreAddress': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1', 'indexed': ['AssetAdded(Address,str,bool)', 'cxbc78363e6ef2cf4bd5a5ac1951a4cec4764f15c6', 'bnUSD', '0x0'], 'data': []} 

{'scoreAddress': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1', 'indexed': ['AssetAd

In [35]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['loans']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)

../../core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1



'cx66f9f3b797d345e9d1001d7214a3040615db6ca1'

In [13]:
send_tx('staking', 0, 'toggleStakingOn', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************


{'txHash': '0x939d5513bd46fe7fd360ee005a34fbf3b701f344cc6bb6495bf55e4d21ae7e59',
 'blockHeight': 252001,
 'blockHash': '0x500fff76ea0de9c74cda447915ab5ffa96300d29485e3cd7e9e589eebd56e595',
 'txIndex': 1,
 'to': 'cx252d78d074053dc19652aa25ad659824fdc31d7e',
 'stepUsed': 130680,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 130680,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [14]:
send_tx('governance', 0, 'balanceToggleStakingEnabled', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling balanceToggleStakingEnabled, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************


{'txHash': '0x5e9675035478e23b1fd7134e891fd519ba0b40db929c7135fcee736327cfaa27',
 'blockHeight': 252003,
 'blockHash': '0xe957d4c1d210e0bc17a0fd946f627d0d04618f535980b5731fc163b556530e09',
 'txIndex': 1,
 'to': 'cx1356633adedaab940853cd87450c70472ffb30e7',
 'stepUsed': 158080,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 158080,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [37]:
# 1.no bad debt for asset, retired from account user2 without a position on Balanced.
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'pos_id': '0x1', 'created': '0x5c09c14d5dc54', 'address': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'snap_id': '0x1', 'snaps_length': '0x1', 'last_snap': '0x1', 'first day': '0x1', 'assets': {'sICX': '0x6c6b935b8bbd400000', 'bnUSD': '0x1b6048686534440000'}, 'total_debt': '0x105ea492156ae494e3', 'collateral': '0x6c6b935b8bbd400000', 'ratio': '0x5bea6d271685b702', 'standing': 'Mining'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
-------------

{'message': 'That address has no outstanding loans or deposited collateral.'}

In [18]:
call_tx('loans', 'getMaxRetireAmount', {'_symbol': 'bnUSD'})


------------------------------------------------------------------------------------------------------------------
Reading getMaxRetireAmount, with parameters {'_symbol': 'bnUSD'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0x466f5c62c9356d58


'0x466f5c62c9356d58'

In [16]:
send_tx('loans', 2000*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 500 * ICX}, user1)


------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 500000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1', 'cx252d78d074053dc19652aa25ad659824fdc31d7e', '0x6c6b935b8bbd400000'], 'data': []} 

{'scoreAddress': 'cxb2d986916fe48a424554526baf61c0451222a067', 'indexed': ['Mint(Address,int,bytes)', 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1'], 'data': ['0x6c6b935b8bbd400000', '0x4e6f6e65']} 

{'scoreAddress': 'cxb2d986916fe48a424554526baf61c0451222a067', 'indexed': ['Transfer(Address,Addre

{'txHash': '0x70b648f216c9deb00887bb4c78dd01a9f2f0f06c318b0a9ee67e7f7a4f6292e7',
 'blockHeight': 252007,
 'blockHash': '0x06b337c93cecac2dbefc6a3970accd1f19ad229c276ce779dcce50683b04197c',
 'txIndex': 1,
 'to': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1',
 'stepUsed': 954540,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 954540,
 'eventLogs': [{'scoreAddress': 'cx66f9f3b797d345e9d1001d7214a3040615db6ca1',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx66f9f3b797d345e9d1001d7214a3040615db6ca1',
    'cx252d78d074053dc19652aa25ad659824fdc31d7e',
    '0x6c6b935b8bbd400000'],
   'data': []},
  {'scoreAddress': 'cxb2d986916fe48a424554526baf61c0451222a067',
   'indexed': ['Mint(Address,int,bytes)',
    'cx66f9f3b797d345e9d1001d7214a3040615db6ca1'],
   'data': ['0x6c6b935b8bbd400000', '0x4e6f6e65']},
  {'scoreAddress': 'cxb2d986916fe48a424554526baf61c0451222a067',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx0000000000000000000000000000000000000000',
    'cx66f9

In [23]:
send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 50 * ICX}, wallet)


------------------------------------------------------------------------------------------------------------------
Calling returnAsset, with parameters {'_symbol': 'bnUSD', '_value': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9', 'indexed': ['Burn(Address,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x2b5e3af16b1880000']} 

{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'hx0000000000000000000000000000000000000000', '0x2b5e3af16b1880000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfeaee564d1cf035aa61184049badb51639fe7107', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'bnUSD', '0x2b5e3af16b1880000'], 'data': ['Loan of 50000000000000000000 bnUSD repaid to Balanced.']} 

{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx2780eeb8c800ac9886786baae281c3d23

{'txHash': '0x052d7d0ca14322080d9046481b0fce424bf4e69b42af76044e8e791be0645c14',
 'blockHeight': 172185,
 'blockHash': '0x71580799e364f02a4d24bdda697bd3663dcf7ba6ff163621ab07a6bcb3ad978b',
 'txIndex': 1,
 'to': 'cxfeaee564d1cf035aa61184049badb51639fe7107',
 'stepUsed': 358340,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 358340,
 'eventLogs': [{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9',
   'indexed': ['Burn(Address,int)',
    'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'],
   'data': ['0x2b5e3af16b1880000']},
  {'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb',
    'hx0000000000000000000000000000000000000000',
    '0x2b5e3af16b1880000'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxfeaee564d1cf035aa61184049badb51639fe7107',
   'indexed': ['LoanRepaid(Address,str,int,str)',
    'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb',
    'bnUSD',
    '0x2b

In [36]:
test_cases = {
        "stories": [
            {
                "description": "User2 tries to retire 10 bnusd",
                  "actions": {
                      "sender":"user2",
                      "first_meth": "transfer",
                      "second_meth":"returnAsset",
                      "deposited_icx":"0",
                      "first_params":{'_to': user2.get_address(), '_value': 20 * ICX},
                      "second_params":{"_symbol": "bnUSD", '_value': 10 * ICX},
                      "expected_status_result":"1"
              }
            }
      ]
    }

for case in test_cases['stories']:
    print(case['description'])
    _to = contracts['bnUSD']['SCORE']
    meth1 = case['actions']['first_meth']
    meth2 = case['actions']['second_meth']
    val = int(case['actions']['deposited_icx'])
    data1 = case['actions']['first_params']
    first_params = {"_to": data1['_to'], "_value": data1['_value']}

    data2 = case['actions']['second_params']
    second_params = {'_symbol': data2['_symbol'], '_value': data2['_value']}

    send_tx('bnUSD', 0 ,meth1 , first_params, user1)
    sleep(2)
    res = send_tx('loans', 0 ,meth2 , second_params, user2)
    assert res['status'] == int(case['actions']['expected_status_result']), 'Retired amount is greater than the current maximum allowed'
    print('Test case passed')

User2 tries to retire 10 bnusd
------------------------------------------------------------------------------------------------------------------
Calling transfer, with parameters {'_to': 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '_value': 2000000000000000000} on the bnUSD contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxbc78363e6ef2cf4bd5a5ac1951a4cec4764f15c6', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '0x1bc16d674ec80000'], 'data': ['0x4e6f6e65']} 

------------------------------------------------------------------------------------------------------------------
Calling returnAsset, with parameters {'_symbol': 'bnUSD', '_value': 10000000000000000000} on the loans contract.
--

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 0 **************************************************
Failure: {'code': 1, 'message': "('borrowers_LINKED_LISTDB', 1)"}


AssertionError: Retired amount is greater than the current maximum allowed